# Build and Train a Model
In this notebook we preprocess the data to get it ready for a model, extend a pre-trained model from Keras, train the model on
the x-ray images in our dataset, and evaluate model performance. The goal is the be able to accurately classify pneumonia with a
reasonable level of accuracy. A good baseline accuracy for this task is ?, we hope to come close to this baseline
and potentially be able to outperform it.

TODO: Find the baseline accuracy from the article

# Import Necessary Packages

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from random import sample
from sklearn import model_selection
import tensorflow as tf
from skimage import io
from keras.preprocessing.image import ImageDataGenerator

## Setup the Data
In this section we setup the data to get it ready for our model. I parse the `Finding Label` to make create binary
classification labels for each type of disease for every observation. I also create a `path` column for each `Image Index`
so I can get to the location of the actual images.

In [ ]:
# Read in the Data
all_xray_df = pd.read_csv('/data/Data_Entry_2017.csv')

# Convert `Finding Labels` to binary labels for each disease
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
for label in all_labels:
    if len(label)>1:
        all_xray_df[label] = all_xray_df['Finding Labels'].map(lambda diseases: 1.0 if label in diseases else 0)

# Create 'path' column for each 'Image Index'
all_image_paths = {os.path.basename(x): x for x in
                   glob(os.path.join('/data','images*', '*', '*.png'))}
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)

# Check the results
print(f'Images Found: {len(all_image_paths)}, Total Observations: {all_xray_df.shape[0]}')
all_xray_df.sample(3)



## Create Training and Validation Sets
In this section I create the training and validation sets for the model. I am using an 90/10 split of the data with
90% of the positive pneumonia cases in the training set and 10% of the positive pneumonia cases in the validation set.
Also for the training set the proportion of positive to negative pneumonia cases will be 50/50.

In [2]:
def create_splits(df, validation_percent):
    # create original train / validation split
    train_data, val_data = model_selection.train_test_split(df, test_size=validation_percent, stratify=df['Pneumonia'])

    # Update training set to be 50/50 split
    pos_indexes = train_data[train_data.Pneumonia == 1].index.tolist()
    neg_indexes = train_data[train_data.Pneumonia == 0].index.tolist()
    neg_sample = sample(neg_indexes, len(pos_indexes))
    train_data = train_data.loc[pos_indexes+neg_sample]

    # Check splits
    print(f'Total Pneumonia Cases: {df[df.Pneumonia==1].shape[0]} \
    {(1-validation_percent)*100}% Pneumonia Cases: {int(df[df.Pneumonia==1].shape[0]*(1-validation_percent))} \
    {validation_percent*100}% Pneumonia Cases: {int(df[df.Pneumonia==1].shape[0]*validation_percent)}')
    print(f'Pneumonia Cases in Training set: {train_data[train_data.Pneumonia==1].shape[0]} \
    Pneumonia Cases in Validation set: {val_data[val_data.Pneumonia==1].shape[0]}')

    print(f'Train Set Size: {train_data.shape[0]}, \
    Pos %: {train_data[train_data.Pneumonia==1].shape[0] / train_data.shape[0] *100}, \
    Neg %: {train_data[train_data.Pneumonia==0].shape[0] / train_data.shape[0] *100}')

    print(f'Original Set Size: {df.shape[0]} \
    Pos %: {df[df.Pneumonia==1].shape[0] / df.shape[0] *100} \
    Neg %: {df[df.Pneumonia==0].shape[0] / df.shape[0] *100}')

    print(f'Validation Set Size: {val_data.shape[0]}, \
    Pos %: {val_data[val_data.Pneumonia == 1].shape[0] / val_data.shape[0] *100}, \
    Neg % {val_data[val_data.Pneumonia == 0].shape[0] / val_data.shape[0] *100}')

    return train_data, val_data

train_df, val_df = create_splits(all_xray_df, 0.1)


NameError: name 'all_xray_df' is not defined

## Implement Image Augmentation and Data Generators
In this section I set up image augmentation to generate a more diverse training set, and create training and validation
set generators to use for model training and testing.

In [ ]:
IMG_SIZE = (256, 256)

# Create Image Data Generator for training set
train_idg = ImageDataGenerator(rescale=1.0/255.0,
                         horizontal_flip = True,
                         vertical_flip = False,
                         rotation_range = 0.5,
                         shear_range = 0.1,
                         zoom_range = 0.15)

train_generator = train_idg.flow_from_dataframe(dataframe=train_df, directory=None,
                                          x_col='path', y_col='Pneumonia',
                                          class_mode = 'binary', target_size=IMG_SIZE, batch_size=24)

# Create a generator for the test set
validation_idg = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = validation_idg.flow_from_dataframe(dataframe=val_df, directory=None,
                                                      x_col='path', y_col='Pneumonia',
                                                      class_mode='binary', target_size=IMG_SIZE, batch_size=1024)

In [ ]:
# Check a random batch of training and validation data to make sure everything looks okay
# Training Examples
t_x, t_y = next(train_generator)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    if c_y == 1: 
        c_ax.set_title('Pneumonia')
    else:
        c_ax.set_title('No Pneumonia')
    c_ax.axis('off')

# Validation Examples
t_x, t_y = next(validation_generator)
fig, m_axs = plt.subplots(2, 2, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone')
    if c_y == 1:
        c_ax.set_title('Pneumonia')
    else:
        c_ax.set_title('No Pneumonia')
    c_ax.axis('off')


## Extending a Pre-trained Model

In [ ]:
# TODO: Write this method for loading in VGG16 (Maybe make modular to load in other pretrained models
def load_pretrained_model(vargs):
    
    # model = VGG16(include_top=True, weights='imagenet')
    # transfer_layer = model.get_layer(lay_of_interest)
    # vgg_model = Model(inputs = model.input, outputs = transfer_layer.output)

    return vgg_model


In [ ]:
# TODO: write model architecture
def build_my_model(vargs):
    
    # my_model = Sequential()
    # ....add your pre-trained model, and then whatever additional layers you think you might
    # want for fine-tuning (Flatteen, Dense, Dropout, etc.)
    
    # if you want to compile your model within this function, consider which layers of your pre-trained model, 
    # you want to freeze before you compile 
    
    # also make sure you set your optimizer, loss function, and metrics to monitor

    return my_model



## STAND-OUT Suggestion: choose another output layer besides just the last classification layer of your modele
## to output class activation maps to aid in clinical interpretation of your model's results

In [ ]:
# TODO: Understand how this works and set it up
## Below is some helper code that will allow you to add checkpoints to your model,
## This will save the 'best' version of your model by comparing it to previous epochs of training

## Note that you need to choose which metric to monitor for your model's 'best' performance if using this code. 
## The 'patience' parameter is set to 10, meaning that your model will train for ten epochs without seeing
## improvement before quitting

# Todo

# weight_path="{}_my_model.best.hdf5".format('xray_class')

# checkpoint = ModelCheckpoint(weight_path, 
#                              monitor= CHOOSE_METRIC_TO_MONITOR_FOR_PERFORMANCE, 
#                              verbose=1, 
#                              save_best_only=True, 
#                              mode= CHOOSE_MIN_OR_MAX_FOR_YOUR_METRIC, 
#                              save_weights_only = True)

# early = EarlyStopping(monitor= SAME_AS_METRIC_CHOSEN_ABOVE, 
#                       mode= CHOOSE_MIN_OR_MAX_FOR_YOUR_METRIC, 
#                       patience=10)

# callbacks_list = [checkpoint, early]

### Train the Model

In [ ]:
# TODO: Setup model training

# history = my_model.fit_generator(train_gen, 
#                           validation_data = (valX, valY), 
#                           epochs = , 
#                           callbacks = callbacks_list)

##### Evaluate Performance of Model

Note, these figures will come in handy for your FDA documentation later in the project

In [ ]:
## After training, make some predictions to assess your model's overall performance
## Note that detecting pneumonia is hard even for trained expert radiologists, 
## so there is no need to make the model perfect.
my_model.load_weights(weight_path)
pred_Y = new_model.predict(valX, batch_size = 32, verbose = True)

In [ ]:
def plot_auc(t_y, p_y):
    
    ## Hint: can use scikit-learn's built in functions here like roc_curve
    
    # Todo
    
    return

## what other performance statistics do you want to include here besides AUC? 


# def ... 
# Todo

# def ...
# Todo
    
#Also consider plotting the history of your model training:

def plot_history(history):
    
    # Todo
    return

In [ ]:
## plot figures

# Todo

Once you feel you are done training, you'll need to decide the proper classification threshold that optimizes your model's performance for a given metric (e.g. accuracy, F1, precision, etc.  You decide) 

In [ ]:
## Find the threshold that optimize your model's performance,
## and use that threshold to make binary classification. Make sure you take all your metrics into consideration.

# Todo

In [ ]:
## Let's look at some examples of predicted v. true with our best model: 

# Todo

# fig, m_axs = plt.subplots(10, 10, figsize = (16, 16))
# i = 0
# for (c_x, c_y, c_ax) in zip(valX[0:100], testY[0:100], m_axs.flatten()):
#     c_ax.imshow(c_x[:,:,0], cmap = 'bone')
#     if c_y == 1: 
#         if pred_Y[i] > YOUR_THRESHOLD:
#             c_ax.set_title('1, 1')
#         else:
#             c_ax.set_title('1, 0')
#     else:
#         if pred_Y[i] > YOUR_THRESHOLD: 
#             c_ax.set_title('0, 1')
#         else:
#             c_ax.set_title('0, 0')
#     c_ax.axis('off')
#     i=i+1

In [ ]:
## Just save model architecture to a .json:

model_json = my_model.to_json()
with open("my_model.json", "w") as json_file:
    json_file.write(model_json)